In [4]:
import pandas as pd
df1=pd.read_csv('03_Customer_Behavior_Data_THPT.csv')
df2=pd.read_csv('03_Item_information_Data_THPT.csv')
df3=pd.read_csv('03_Shelf_Information_Data_THPT.csv')
df_temp = pd.merge(df1, df2, on='Shelf ID', how='inner')
df_master = pd.merge(df_temp, df3, on='Shelf ID', how='inner')
print(df_master['Item ID_x'].unique())
print(df_master['Item ID_y'].unique())
print(df_master.columns)
print(df_master.head())

[16  5  4  8  6 11  1 15  2  9 10 12 17  7  3  0 14 24 23 22 13 18 19 20
 21]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]
Index(['Shelf ID', 'Person ID', 'Date', 'Time', 'Day', 'Age', 'Gender',
       'Height_x', 'Weight', 'Married status', 'Item ID_x',
       'Looking at item (s)', 'Holding the item (s)', 'Holding the bag',
       'Picking up item', 'Returning item', 'Putting item into bag',
       'Taking item out of bag', 'Putting item into bag in the 2nd time',
       'Item ID_y', 'Name', 'Origin', 'Price', 'Marketing strategy',
       'Discount ', 'Rate', 'Description', 'Width', 'Height_y', 'Length',
       'Number of items', 'Shape'],
      dtype='object')
   Shelf ID  Person ID        Date      Time       Day  Age  Gender  Height_x  \
0         4          1  2024-07-06  10:00:00  Saturday   52  Female       162   
1         4          1  2024-07-06  10:00:00  Saturday   52  Female       162   
2         4          1  2024-07-06  10:00:00  Saturd

In [5]:
#câu hỏi 1: Thống kê 5 mặt hàng với lượng khách hàng nhìn thấy nhiều nhất
group_looked = df_master.groupby('Item ID_x')['Looking at item (s)'].sum().reset_index(name='Tổng lượt nhìn')
top_5_items_looked = group_looked.sort_values(by='Tổng lượt nhìn', ascending=False).head(5)
print(top_5_items_looked)

   Item ID_x  Tổng lượt nhìn
6          6          577406
7          7          569839
1          1          464091
8          8          461510
0          0          452643


In [6]:
#câu hỏi 2: Thống kê 5 mặt hàng có tổng thời gian nhìn và cầm xem lâu nhất
df_master['total time (s)']=df_master['Looking at item (s)'] + df_master['Holding the item (s)']
grouped = df_master.groupby('Item ID_x')['total time (s)'].sum().reset_index(name='total (s)')
top_5_items = grouped.sort_values(by='total (s)', ascending=False).head(5)
print(top_5_items)

   Item ID_x  total (s)
6          6    1095731
7          7    1078667
1          1     905554
0          0     875481
8          8     844727


In [7]:
#câu hỏi 3: Thống kê 5 mặt hàng được mua nhiều nhất
df_master['Buy']=df_master['Putting item into bag in the 2nd time']==True
grouped = df_master.groupby('Item ID_x')['Buy'].sum().reset_index(name='Buys')
top_5_items = grouped.sort_values(by='Buys', ascending=False).head(5)
print(top_5_items)

   Item ID_x  Buys
1          1  6804
0          0  6720
6          6  6675
2          2  6476
7          7  6325


In [8]:
#câu hỏi 4: Thống kê 5 mặt hàng thường được cầm lên rồi trả lại nhiều nhất
df_master['Return']=(df_master['Picking up item']==True) & (df_master['Returning item']==True)
grouped = df_master.groupby('Item ID_x')['Return'].sum().reset_index(name='Return times')
top_5_items = grouped.sort_values(by='Return times', ascending=False).head(5)
print(top_5_items)

   Item ID_x  Return times
6          6          7800
5          5          7657
8          8          7520
7          7          7416
3          3          6770


In [9]:
#câu hỏi 5: Các nhóm khách hàng theo độ tuổi (Thiếu niên: 18 - 30; Trung niên: 31 - 60; Cao tuổi: > 60) mua mặt hàng nào nhiều nhất?
df_master['Thiếu niên'] = (df_master['Age'] >= 18) & (df_master['Age'] <= 30)
df_master['Trung niên'] =(df_master['Age']>=31) & (df_master['Age']<=60)
df_master['Cao tuổi'] =df_master['Age']>60
set=['Thiếu niên','Trung niên','Cao tuổi']
for i in set:
    grouped = df_master.groupby('Item ID_x')[i].sum().reset_index(name=i)
    top_1_items = grouped.sort_values(by=i, ascending=False).head(1)
    print(top_1_items)

   Item ID_x  Thiếu niên
1          1        4025
   Item ID_x  Trung niên
7          7        8260
   Item ID_x  Cao tuổi
8          8      4526


In [10]:
#câu hỏi 6: Ngày nào trong tuần có doanh thu cao nhất?
grouped = df_master.groupby('Day')['Item ID_x'].sum().reset_index(name='item_quantity')
top_1_items = grouped.sort_values(by='item_quantity', ascending=False).head(1)
print(top_1_items)

        Day  item_quantity
2  Saturday         618017


In [ ]:
#câu hỏi 7: Trong 3 nhóm tuổi sau: Thiếu niên (18 - 30),  Trung niên (31 - 60), Cao tuổi: (> 60),  nhóm tuổi nào có số người đi siêu thị nhiều nhất?
group = df_master.groupby('Person ID').size().reset_index(name='số lượt xuất hiện')
group = group.merge(df_master[['Person ID', 'Age']].drop_duplicates(), on='Person ID')

def classify_age(age):
    if 18 <= age <= 30:
        return 'Thiếu niên'
    elif 31 <= age <= 60:
        return 'Trung niên'
    else:
        return 'Cao tuổi'

group['Nhóm tuổi'] = group['Age'].apply(classify_age)
result = group.groupby('Nhóm tuổi')['số lượt xuất hiện'].sum().reset_index()
max_group = result.loc[result['số lượt xuất hiện'].idxmax()]
print(result)
print(f"Nhóm tuổi có số người đi siêu thị nhiều nhất là: {max_group['Nhóm tuổi']} với {max_group['số lượt xuất hiện']} lượt xuất hiện.")

    Nhóm tuổi  số lượt xuất hiện
0    Cao tuổi              68081
1  Thiếu niên              57558
2  Trung niên             123187
Nhóm tuổi có số người đi siêu thị nhiều nhất là: Trung niên với 123187 lượt xuất hiện.


In [ ]:
print(df_master.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248826 entries, 0 to 248825
Data columns (total 37 columns):
 #   Column                                 Non-Null Count   Dtype         
---  ------                                 --------------   -----         
 0   Shelf ID                               248826 non-null  int64         
 1   Person ID                              248826 non-null  int64         
 2   Date                                   15394 non-null   datetime64[ns]
 3   Time                                   248826 non-null  object        
 4   Day                                    248826 non-null  object        
 5   Age                                    248826 non-null  int64         
 6   Gender                                 248826 non-null  object        
 7   Height_x                               248826 non-null  int64         
 8   Weight                                 248826 non-null  int64         
 9   Married status                         248826 no

In [ ]:
#câu hỏi 8: Top 5 các mặt hàng giảm giá được người dùng mua nhiều nhất?
df_master['BuywithDiscount']=(df_master['Putting item into bag in the 2nd time']==True)&(df_master['Discount ']>0)
grouped = df_master.groupby('Item ID_x')['BuywithDiscount'].sum().reset_index(name='BuywithDiscount')
top_5_items = grouped.sort_values(by='BuywithDiscount', ascending=False).head(5)
print(top_5_items)

    Item ID_x  BuywithDiscount
0           0             2707
6           6             2679
11         11             2609
8           8             2569
1           1             2567


In [ ]:
#câu hỏi 9: Top 3 quầy hàng nào có lượt cầm xem nhiều nhất?
group_holding = df_master.groupby('Shelf ID')['Holding the item (s)'].sum().reset_index(name='Tổng lượt cầm xem')
top_3_sections_holding = group_holding.sort_values(by='Tổng lượt cầm xem', ascending=False).head(3)
print(top_3_sections_holding)

   Shelf ID  Tổng lượt cầm xem
7         7            1603104
1         1            1056024
4         4             976467


In [ ]:
#câu hỏi 10: Trong số các sản phẩm được đánh giá cao nhất, top 3 sản phẩm nào được mua nhiều nhất?
df_master['Buy']=df_master['Putting item into bag in the 2nd time']==True
highly_rated_items = df_master[df_master['Rate'] > 4]
grouped = highly_rated_items.groupby('Item ID_y')['Buy'].sum().reset_index(name='Total_Purchases')
top_3_items = grouped.sort_values(by='Total_Purchases', ascending=False).head(3)
print(top_3_items)

   Item ID_y  Total_Purchases
0          0             3216
5          6             2705
8         10             2414


In [19]:
#câu hỏi 11: Câu hỏi mở: Tìm ra mặt hàng hay được chú ý nhất của những nhóm khách hàng cụ thể
groups = ['Thiếu niên', 'Trung niên', 'Cao tuổi']
for group in groups:
    grouped = df_master.groupby([group, 'Name'])['Looking at item (s)'].sum().reset_index(name='Total_Time_Looked')
    top_item_per_group = grouped.loc[grouped.groupby(group)['Total_Time_Looked'].idxmax()]
    print(f"Mặt hàng được chú ý nhất trong nhóm: {group}")
    print(top_item_per_group[['Name', 'Total_Time_Looked']])

Mặt hàng được chú ý nhất trong nhóm: Thiếu niên
             Name  Total_Time_Looked
3    Bibigo Mandu              78394
137  Bibigo Mandu              23050
Mặt hàng được chú ý nhất trong nhóm: Trung niên
             Name  Total_Time_Looked
3    Bibigo Mandu              51259
137  Bibigo Mandu              50185
Mặt hàng được chú ý nhất trong nhóm: Cao tuổi
             Name  Total_Time_Looked
3    Bibigo Mandu              73235
137  Bibigo Mandu              28209
